# Temporal Widget — Novelty → Tempogram → Beat Track → Beats & Bars

Interactive pipeline for a single audio file.

- **Split widgets** (one per notebook): open **novelty_widget.ipynb**, **tempogram_widget.ipynb**, **beats_widget.ipynb**. Each loads with default data from the preceding step; use the Override cell in each to feed custom inputs. Preceding params = re-run that notebook/cell; each widget's params = live or Recompute.
- **Combined widget** (below): single pipeline in this notebook; only zoom is live, everything else on Recompute.

All compute isolated under `notebooks/scratch/widget/`.

In [1]:
import sys
from pathlib import Path

import ipywidgets as widgets
import librosa
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, Javascript, display

# Ensure widget dir and project root on path
widget_dir = Path.cwd()
for candidate in [
    Path.cwd(),
    Path.cwd() / "widget",
    Path.cwd() / "scratch" / "widget",
    Path.cwd() / "notebooks" / "scratch" / "widget",
]:
    if (candidate / "temporal_widget_helpers.py").exists():
        widget_dir = candidate
        break
sys.path.insert(0, str(widget_dir))

# Project root for dijon
project_root = widget_dir
for _ in range(6):
    if (project_root / "src" / "dijon").exists():
        break
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

from temporal_widget_helpers import (
    DEFAULT_TEMPO_MIN,
    DEFAULT_TEMPO_MAX,
    build_audio_with_waveform_html,
    FS_NOV,
    GAMMA_OPTIONS,
    HOP_OPTIONS,
    HOP_SEC_OPTIONS,
    LOCAL_M_OPTIONS,
    NOVELTY_DEFAULTS,
    WINDOW_SEC_OPTIONS,
    build_beat_sonification_audio,
    compute_tempogram,
    eager_compute_all,
    get_head_in_time,
    get_novelty_for_method,
    get_tempogram_for_methods,
    load_audio_and_markers,
    novelty_params_key,
    run_beat_tracking,
    run_beats_and_bars,
    tempo_from_tempogram,
    zoom_level_to_half_sec,
)

%matplotlib inline

In [2]:
TRACK = "YTB-005"
MARKER = ""  # "" or None = full track; else e.g. "HEAD_IN", "LICK01"

x, sr, seg_start, seg_end, marker_label = load_audio_and_markers(TRACK, MARKER or None)
duration_s = len(x) / sr
print(f"Track: {TRACK} | Marker: {marker_label} | Duration: {duration_s:.2f}s @ {sr} Hz")

Track: YTB-005 | Marker: FULL | Duration: 175.48s @ 22050 Hz


In [3]:
# Eager precompute: novelty + tempogram on discrete grid (load-heavy, interaction-light)
novelty_cache, tempogram_cache, init_time, init_timings = eager_compute_all(x, sr)
print(f"Eager init: {init_time:.2f}s ({len(novelty_cache)} novelty + {len(tempogram_cache)} tempogram variants)")
for k, v in sorted(init_timings.items(), key=lambda p: (0 if p[0] == "total" else 1, p[0])):
    if k != "total":
        print(f"  {k}: {v:.3f}s")

Eager init: 103.77s (896 novelty + 0 tempogram variants)
  novelty_all: 103.769s


**Split widgets (separate notebooks)**  
Open and run in order: **novelty_widget.ipynb** → **tempogram_widget.ipynb** → **beats_widget.ipynb**. Each has its own Override cell and loads with default data from the preceding step.

In [ ]:
# Split widgets live in novelty_widget.ipynb, tempogram_widget.ipynb, beats_widget.ipynb (run those for the cascading split UI).
# Below: combined widget only (single pipeline in this notebook).

In [ ]:
# Novelty split widget → see novelty_widget.ipynb
pass

In [ ]:
# Tempogram split widget → see tempogram_widget.ipynb
pass

In [ ]:
# Beats split widget → see beats_widget.ipynb
pass

HTML(value='<h3>Split: Novelty | Tempogram | Beats</h3>')

In [ ]:
# --- Control widgets ---
# Labels via _label() above; minimal/empty descriptions to avoid clipping
_compact = dict(layout=widgets.Layout(margin="0 2px 2px 2px"))

novelty_method = widgets.Dropdown(options=["energy", "spectral", "phase", "complex"], value="spectral", description="Novelty:", **_compact)
tempogram_method = widgets.Dropdown(options=["fourier", "autocorr", "cyclic"], value="fourier", description="Tempogram:", **_compact)

hop_sel = widgets.Dropdown(options=HOP_OPTIONS, value=256, description="", **_compact)
gamma_sel = widgets.Dropdown(options=GAMMA_OPTIONS, value=100.0, description="", **_compact)
local_avg_sel = widgets.Dropdown(options=LOCAL_M_OPTIONS, value=10, description="", **_compact)

window_sec_sel = widgets.Dropdown(options=WINDOW_SEC_OPTIONS, value=5.0, description="", **_compact)
hop_sec_sel = widgets.Dropdown(options=HOP_SEC_OPTIONS, value=0.1, description="", **_compact)

factor_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description="", **_compact)
tempo_override = widgets.FloatText(value=0, description="", placeholder="0=auto", **_compact)
beats_per_bar_sel = widgets.Dropdown(options=[("Auto", 0), ("2", 2), ("3", 3), ("4", 4)], value=0, description="", **_compact)

zoom_enabled = widgets.Checkbox(value=False, description="Zoom on playhead", **_compact)
zoom_level = widgets.IntSlider(value=1, min=1, max=10, step=1, description="", **_compact)

sonify_toggle = widgets.Checkbox(value=False, description="Sonify beats", **_compact)

def _label(txt):
    return widgets.HTML(f'<div style="font-size:9px;margin-top:2px;margin-bottom:0;line-height:1.2">{txt}</div>')

def _label_def(txt, recompute=False):
    color = "#1565c0" if recompute else "inherit"
    return widgets.HTML(f'<div style="font-size:9px;margin-top:2px;margin-bottom:0;line-height:1.2;color:{color}">{txt}</div>')

_half = widgets.Layout(flex="1 1 45%", min_width="80px")

# Factory defaults (used by "Reset to factory")
FACTORY_DEFAULTS = {
    "novelty_method": "spectral", "tempogram_method": "fourier", "hop": 256,
    "gamma": 100.0, "local_avg": 10, "window_sec": 5.0, "hop_sec": 0.1,
    "factor": 1.0, "tempo_override": 0, "beats_per_bar": 0,
    "zoom_enabled": False, "zoom_level": 1, "sonify_toggle": False,
}

# Editable defaults (load from factory; user can tweak)
def _make_default_widgets():
    d = dict(FACTORY_DEFAULTS)
    return {
        "novelty_method": widgets.Dropdown(options=["energy", "spectral", "phase", "complex"], value=d["novelty_method"], **_compact),
        "tempogram_method": widgets.Dropdown(options=["fourier", "autocorr", "cyclic"], value=d["tempogram_method"], **_compact),
        "hop": widgets.Dropdown(options=HOP_OPTIONS, value=d["hop"], **_compact),
        "gamma": widgets.Dropdown(options=GAMMA_OPTIONS, value=d["gamma"], **_compact),
        "local_avg": widgets.Dropdown(options=LOCAL_M_OPTIONS, value=d["local_avg"], **_compact),
        "window_sec": widgets.Dropdown(options=WINDOW_SEC_OPTIONS, value=d["window_sec"], **_compact),
        "hop_sec": widgets.Dropdown(options=HOP_SEC_OPTIONS, value=d["hop_sec"], **_compact),
        "factor": widgets.FloatText(value=d["factor"], **_compact),
        "tempo_override": widgets.FloatText(value=d["tempo_override"], **_compact),
        "beats_per_bar": widgets.Dropdown(options=[("Auto", 0), ("2", 2), ("3", 3), ("4", 4)], value=d["beats_per_bar"], **_compact),
        "zoom_enabled": widgets.Checkbox(value=d["zoom_enabled"], **_compact),
        "zoom_level": widgets.IntSlider(value=d["zoom_level"], min=1, max=10, **_compact),
        "sonify_toggle": widgets.Checkbox(value=d["sonify_toggle"], **_compact),
    }
default_widgets = _make_default_widgets()

recompute_btn = widgets.Button(description="Recompute", button_style="primary")
status_out = widgets.Output()
plot_out = widgets.Output()
audio_bot_out = widgets.Output()
_pipeline_state = {}
# Combined widget uses its own caches; no pre-compute. Only zoom is live; everything else computed on Recompute.
_combined_novelty_cache = {}
_combined_tempogram_cache = {}

In [8]:
def _sync_novelty_params(method):
    d = NOVELTY_DEFAULTS.get(method, NOVELTY_DEFAULTS["spectral"])
    hop_sel.value = d["H"] if d["H"] in [o[1] for o in HOP_OPTIONS] else 256
    g = d.get("gamma")
    gamma_sel.value = g if g is not None and g in [o[1] for o in GAMMA_OPTIONS] else 100.0
    m = d.get("M", 0)
    local_avg_sel.value = m if m in [o[1] for o in LOCAL_M_OPTIONS] else 10

def on_novelty_change(change):
    if change["name"] == "value":
        _sync_novelty_params(change["new"])

novelty_method.observe(on_novelty_change, names="value")

In [9]:
TARGET_FAST_MS = 250

def _run_full_pipeline():
    import time
    t0 = time.perf_counter()
    nm, tm = novelty_method.value, tempogram_method.value
    N = NOVELTY_DEFAULTS[nm]["N"]
    gamma_val = gamma_sel.value if NOVELTY_DEFAULTS[nm].get("gamma") is not None else None
    nov, nov_100 = get_novelty_for_method(
        _combined_novelty_cache, x, sr, nm, N, hop_sel.value, gamma_val, local_avg_sel.value
    )
    nov_key = novelty_params_key(nm, N, hop_sel.value, gamma_val, local_avg_sel.value)
    t1 = time.perf_counter()
    tempogram, T_coef, F_coef, aux = get_tempogram_for_methods(
        _combined_tempogram_cache, nov_100, nov_key, tm,
        window_sec=window_sec_sel.value, hop_sec=hop_sec_sel.value,
    )
    t2 = time.perf_counter()
    tempo_bpm = float(tempo_override.value) if tempo_override.value and tempo_override.value > 0 else None
    if tempo_bpm is None:
        tempo_bpm = tempo_from_tempogram(tempogram, aux["F_coef_BPM"], tm, aux=aux)
    B, D, P = run_beat_tracking(nov_100, tempo_bpm, factor=factor_slider.value)
    beat_times = B / FS_NOV
    t3 = time.perf_counter()
    head_in = get_head_in_time(TRACK)
    bpb = beats_per_bar_sel.value if beats_per_bar_sel.value != 0 else None
    labels, beats_per_bar, low_energy, high_energy = run_beats_and_bars(
        beat_times, head_in, x, sr, beats_per_bar_override=bpb
    )
    t4 = time.perf_counter()
    _pipeline_state.update({
        "nm": nm, "tm": tm, "nov_100": nov_100, "tempogram": tempogram,
        "T_coef": T_coef, "F_coef": F_coef, "aux": aux, "B": B, "beat_times": beat_times,
        "labels": labels, "beats_per_bar": beats_per_bar, "tempo_bpm": tempo_bpm,
    })
    return t1 - t0, t2 - t1, t3 - t2, t4 - t3, t4 - t0

def _run_fast_path():
    import time
    t0 = time.perf_counter()
    s = _pipeline_state
    if not s:
        return _run_full_pipeline()
    nov_100, tempogram, aux = s["nov_100"], s["tempogram"], s["aux"]
    tm, tempo_bpm = s["tm"], s["tempo_bpm"]
    tempo_bpm = float(tempo_override.value) if tempo_override.value and tempo_override.value > 0 else tempo_bpm
    B, D, P = run_beat_tracking(nov_100, tempo_bpm, factor=factor_slider.value)
    beat_times = B / FS_NOV
    t1 = time.perf_counter()
    head_in = get_head_in_time(TRACK)
    bpb = beats_per_bar_sel.value if beats_per_bar_sel.value != 0 else None
    labels, beats_per_bar, _, _ = run_beats_and_bars(beat_times, head_in, x, sr, beats_per_bar_override=bpb)
    t2 = time.perf_counter()
    _pipeline_state.update({"B": B, "beat_times": beat_times, "labels": labels, "beats_per_bar": beats_per_bar, "tempo_bpm": tempo_bpm})
    return 0, 0, t1 - t0, t2 - t1, t2 - t0

def _redraw_plots(center_time=None, update_audio=True):
    s = _pipeline_state
    if not s:
        return
    nm, tm = s["nm"], s["tm"]
    nov_100, tempogram, T_coef, F_coef, B, beat_times = s["nov_100"], s["tempogram"], s["T_coef"], s["F_coef"], s["B"], s["beat_times"]
    half = zoom_level_to_half_sec(zoom_level.value, duration_s) if zoom_enabled.value else duration_s
    xlo = max(0, (center_time or 0) - half) if zoom_enabled.value and center_time is not None else 0
    xhi = min(duration_s, (center_time or 0) + half) if zoom_enabled.value and center_time is not None else duration_s
    if not zoom_enabled.value:
        xlo, xhi = 0, duration_s
    with plot_out:
        plot_out.clear_output(wait=True)
        # 5 plots: waveform (zoom), novelty, tempogram, beats, bars
        fig, axes = plt.subplots(5, 1, figsize=(12, 9), sharex=True)
        t_wav = np.arange(len(x)) / sr
        axes[0].plot(t_wav, x, color="#888", linewidth=0.5)
        axes[0].set_ylabel("Amp")
        axes[0].set_title("Waveform")
        axes[0].set_xlim(xlo, xhi)
        t_nov = np.arange(len(nov_100)) / FS_NOV
        axes[1].plot(t_nov, nov_100, "k", linewidth=0.7)
        axes[1].set_ylabel("Novelty")
        axes[1].set_title(f"Novelty ({nm})")
        axes[1].set_xlim(xlo, xhi)
        extent = [T_coef[0], T_coef[-1], F_coef[0], F_coef[-1]]
        axes[2].imshow(tempogram, aspect="auto", origin="lower", extent=extent, cmap="gray_r")
        axes[2].set_ylabel("Tempo (BPM)" if tm != "cyclic" else "Scale")
        axes[2].set_title(f"Tempogram ({tm})")
        axes[2].set_xlim(xlo, xhi)
        axes[3].plot(t_nov, nov_100, "k", linewidth=0.7)
        if len(B) > 0:
            axes[3].stem(beat_times, nov_100[B], linefmt="r-", markerfmt="ro", basefmt=" ")
        axes[3].set_ylabel("Novelty")
        axes[3].set_title("Beats")
        axes[3].set_xlim(xlo, xhi)
        axes[4].plot(beat_times, np.arange(len(beat_times)), "k.", markersize=2)
        axes[4].set_ylabel("Beat index")
        axes[4].set_xlabel("Time (s)")
        axes[4].set_title("Bars")
        axes[4].set_xlim(xlo, xhi)
        plt.tight_layout()
        plt.show()
    if update_audio:
        with audio_bot_out:
            audio_bot_out.clear_output(wait=True)
            x_play = build_beat_sonification_audio(x, sr, beat_times, s["labels"], s["beats_per_bar"], get_head_in_time(TRACK)) if sonify_toggle.value else x
            h_bot = build_audio_with_waveform_html(x, x_play, sr, max_duration_sec=max(60, duration_s))
            display(HTML('<div style="font-size:10px">Waveform: original. Audio: ' + ('sonified' if sonify_toggle.value else 'original') + '</div>'))
            display(HTML('<div id="combined-w-audio-container">' + h_bot + '</div>'))
            _inject_playhead_sync_js()

def _inject_playhead_sync_js():
    from IPython.display import Javascript, display
    display(Javascript("""
    (function(){
      var c = document.getElementById('combined-w-audio-container');
      var audio = c ? c.querySelector('audio') : null;
      if (!audio) return;
      var lastSent = -1;
      audio.addEventListener('timeupdate', function(){
        var t = audio.currentTime;
        if (Math.abs(t - lastSent) < 0.25) return;
        lastSent = t;
        if (typeof Jupyter !== 'undefined' && Jupyter.notebook && Jupyter.notebook.kernel) {
          Jupyter.notebook.kernel.execute('_zoom_center_from_playhead(' + t + ')');
        }
      });
    })();
    """))

def _zoom_center_from_playhead(t):
    if zoom_enabled.value and t is not None:
        _redraw_plots(center_time=float(t), update_audio=False)

def do_recompute(_, fast_only=False):
    import time
    with status_out:
        status_out.clear_output()
        if fast_only and _pipeline_state:
            tn, tt, tb, tbar, total = _run_fast_path()
            path = "fast"
        else:
            tn, tt, tb, tbar, total = _run_full_pipeline()
            path = "full"
        s = _pipeline_state
        total_ms = total * 1000
        warn = " (over 250ms)" if total_ms > TARGET_FAST_MS else ""
        print(f"[{path}] Novelty: {tn:.2f}s | Tempogram: {tt:.2f}s | Beats: {tb:.2f}s | Bars: {tbar:.2f}s | Total: {total:.2f}s{warn}")
        print(f"Tempo: {s['tempo_bpm']:.1f} BPM | Beats: {len(s['B'])} | B/bar: {s['beats_per_bar']}")
    _redraw_plots(center_time=0)

def on_fast_control_change(_):
    do_recompute(None, fast_only=True)

def on_recompute_param_change(_):
    do_recompute(None, fast_only=False)

recompute_btn.on_click(lambda b: do_recompute(b, fast_only=False))
novelty_method.observe(on_recompute_param_change, names="value")
tempogram_method.observe(on_recompute_param_change, names="value")
hop_sel.observe(on_recompute_param_change, names="value")
gamma_sel.observe(on_recompute_param_change, names="value")
local_avg_sel.observe(on_recompute_param_change, names="value")
window_sec_sel.observe(on_recompute_param_change, names="value")
hop_sec_sel.observe(on_recompute_param_change, names="value")
factor_slider.observe(on_fast_control_change, names="value")
tempo_override.observe(on_fast_control_change, names="value")
beats_per_bar_sel.observe(on_fast_control_change, names="value")
def _on_zoom_change(_):
    if _pipeline_state:
        _redraw_plots(center_time=0, update_audio=False)

zoom_enabled.observe(_on_zoom_change, names="value")
zoom_level.observe(_on_zoom_change, names="value")

def _on_sonify_change(_):
    if _pipeline_state:
        _redraw_plots(center_time=0, update_audio=True)

sonify_toggle.observe(_on_sonify_change, names="value")

def _param_widgets():
    return {
        "novelty_method": novelty_method, "tempogram_method": tempogram_method, "hop": hop_sel,
        "gamma": gamma_sel, "local_avg": local_avg_sel, "window_sec": window_sec_sel,
        "hop_sec": hop_sec_sel,
        "factor": factor_slider, "tempo_override": tempo_override, "beats_per_bar": beats_per_bar_sel,
        "zoom_enabled": zoom_enabled, "zoom_level": zoom_level, "sonify_toggle": sonify_toggle,
    }

def _apply_defaults_to_params():
    pw = _param_widgets()
    for k, w in default_widgets.items():
        try:
            v = w.value
            if v is None:
                continue
            if k in ("gamma", "window_sec", "hop_sec", "factor", "tempo_override"):
                v = float(v)
            elif k in ("local_avg", "zoom_level"):
                v = int(v)
            pw[k].value = v
        except (TypeError, ValueError):
            pass

def _apply_params_to_defaults():
    pw = _param_widgets()
    for k, w in default_widgets.items():
        w.value = pw[k].value

def _apply_factory_to_all():
    for k, w in default_widgets.items():
        w.value = FACTORY_DEFAULTS[k]
    _apply_defaults_to_params()

def on_reset_click(_):
    _apply_defaults_to_params()
    do_recompute(None)

def on_set_default_click(_):
    _apply_params_to_defaults()
    do_recompute(None)

def on_factory_reset_click(_):
    _apply_factory_to_all()
    do_recompute(None)

In [10]:
# Assemble layout: controls on left, outputs on right (wider to avoid value cutoff)
_reset_btn = widgets.Button(description="RESET", layout=widgets.Layout(flex="1"))
_set_default_btn = widgets.Button(description="SET TO DEFAULT", layout=widgets.Layout(flex="1"))
_factory_btn = widgets.Button(description="Reset to factory", button_style="warning", layout=widgets.Layout(width="100%"))
_reset_btn.on_click(on_reset_click)
_set_default_btn.on_click(on_set_default_click)
_factory_btn.on_click(on_factory_reset_click)

def _def_cell(label_html, w):
    return widgets.VBox([label_html, w], layout=_half)

defaults_rows = [
    widgets.HBox([
        _def_cell(_label_def("Novelty", recompute=True), default_widgets["novelty_method"]),
        _def_cell(_label_def("Tempogram", recompute=True), default_widgets["tempogram_method"]),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
    widgets.HBox([
        _def_cell(_label_def("Hop", recompute=True), default_widgets["hop"]),
        _def_cell(_label_def("Gamma", recompute=True), default_widgets["gamma"]),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
    widgets.HBox([
        _def_cell(_label_def("Local M", recompute=True), default_widgets["local_avg"]),
        _def_cell(_label_def("Window (s)", recompute=True), default_widgets["window_sec"]),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
    widgets.HBox([
        _def_cell(_label_def("Hop (s)", recompute=True), default_widgets["hop_sec"]),
        _def_cell(_label_def("Factor", recompute=False), default_widgets["factor"]),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
    widgets.HBox([
        _def_cell(_label_def("Tempo ov", recompute=False), default_widgets["tempo_override"]),
        _def_cell(_label_def("B/bar", recompute=False), default_widgets["beats_per_bar"]),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
    widgets.HBox([
        _def_cell(_label_def("Sonify", recompute=False), default_widgets["sonify_toggle"]),
        widgets.VBox([_label_def("Zoom on", recompute=False), default_widgets["zoom_enabled"]], layout=_half),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
    widgets.HBox([
        _def_cell(_label_def("Zoom level", recompute=False), default_widgets["zoom_level"]),
    ], layout=widgets.Layout(width="100%", justify_content="space-between")),
]

defaults_section = widgets.VBox([
    widgets.HTML("<b>Defaults</b> <span style='font-size:8px;color:#1565c0'>blue=recompute</span>"),
    *defaults_rows,
    widgets.HBox([_reset_btn, _set_default_btn]),
    _factory_btn,
], layout=widgets.Layout(margin="8px 0 0 0"))

controls = widgets.VBox([
    widgets.HTML("<b style='color:#1565c0'>Novelty</b>"),
    novelty_method,
    _label_def("Hop size (frames)", recompute=True), hop_sel,
    _label_def("Log compression gamma", recompute=True), gamma_sel,
    _label_def("Local avg window M", recompute=True), local_avg_sel,
    widgets.HTML("<b style='color:#1565c0'>Tempogram</b>"),
    tempogram_method,
    _label_def("Window (s)", recompute=True), window_sec_sel,
    _label_def("Hop (s)", recompute=True), hop_sec_sel,
    widgets.HTML("<b>Beat / Bars</b>"),
    _label_def("Factor", recompute=False), factor_slider,
    _label_def("Tempo override", recompute=False), tempo_override,
    _label_def("Beats per bar", recompute=False), beats_per_bar_sel,
    sonify_toggle,
    widgets.HTML("<b>Zoom</b>"),
    zoom_enabled,
    _label_def("Zoom (1=out, 10=in)", recompute=False), zoom_level,
    recompute_btn,
    defaults_section,
], layout=widgets.Layout(width="320px", min_width="300px"))
outputs = widgets.VBox([
    status_out,
    plot_out,
    widgets.HTML("<b>Audio (waveform: original)</b>"),
    audio_bot_out,
], layout=widgets.Layout(flex="1"))
dashboard = widgets.HBox([controls, outputs], layout=widgets.Layout(width="100%", padding="0 4px 0 0"))
display(widgets.HTML("<h3>Combined</h3>"))
display(dashboard)

# Trigger initial compute
do_recompute(None)

HTML(value='<h3>Combined</h3>')